In [2]:
import numpy as np
from Bio import AlignIO
import os
import subprocess

In [15]:
def find_nearest(pdb, n_nearest):
    import numpy as np
    from scipy.spatial import distance
    """
    This function finds the n_nearest atoms to a given atom in a pdb file.
    """
    with open(pdb, 'r') as f:
        lines = [line for line in f if line.startswith('ATOM') and line[13:15] == 'CA']
    coords = np.array([line.split()[6:9] for line in lines], dtype=float)
    distance_matrix = distance.cdist(coords, coords)
    nearest_ind = np.array([np.argsort(distance_matrix[i])[1:n_nearest+1] for i in range(len(distance_matrix))])
    nearest_dist = np.array([np.sort(distance_matrix[i])[1:n_nearest+1] for i in range(len(distance_matrix))])
    return nearest_ind, nearest_dist

In [16]:
def jsd(p, q):
    """
    This function calculates the Jensen-Shannon divergence between two probability distributions.
    """
    from scipy.special import rel_entr
    import numpy as np
    m = 0.5 * (p + q)
    jsd = 0.5 * np.sum(rel_entr(p, m)) + 0.5 * np.sum(rel_entr(q, m))
    return jsd

In [ ]:
def mmseqs_conservation(input_pdb : str,input_fasta : str, database_fasta : str, windows:bool = False, max_seqs:int = 1000, include_neighbors:bool=False, n_nearest:int=5, decay:float=0.5, alpha:float=0.5):
    import subprocess
    if not os.path.exists('./data/mmseqs_data'):
        os.makedirs('./data/mmseqs_data')
    subprocess.run(["mmseqs","createdb",database_fasta,"./data/mmseqs_data/database_db"])                #create mmseq db file from fasta (database)
    subprocess.run(["mmseqs","createdb", input_fasta, "./data/mmseqs_data/protein_db"])        #create mmseq db file from fasta (input protein)
    subprocess.run(["mmseqs","search","./data/mmseqs_data/protein_db","./data/mmseqs_data/database_db","./data/mmseqs_data/result","./data/mmseqs_data/tmp","--max-seqs",str(max_seqs)])     #search homolog proteins in database
    subprocess.run(["mmseqs","result2msa","./data/mmseqs_data/protein_db","./data/mmseqs_data/database_db","./data/mmseqs_data/result","./data/mmseqs_data/alignment_msa.fasta"])     # generate multiple sequence alignment from search result
    
    aa_order = ['A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']
    blosum62_background = np.array([0.078, 0.051, 0.041, 0.052, 0.024, 0.034, 0.059, 0.083, 0.025, 0.062, 0.092, 0.056, 0.024, 0.044, 0.043, 0.059, 0.055, 0.014, 0.034, 0.072])
    alignment = AlignIO.read('./data/mmseqs_data/alignment_msa.fasta', 'fasta')   # read msa using biopython
    alignment_array = np.array([list(rec.seq) for rec in alignment], dtype = 'U1')
    prot_seq = alignment[0].seq    # first sequence is protein of interest
    columns = np.where(np.array(prot_seq) != '-')[0]
    alignment_array_filtered = alignment_array[:, columns]
    aa_prop_array = np.zeros((20, alignment_array_filtered.shape[1]))   # initiate array to store amino acid proportions
    for n_col, col in enumerate(alignment_array_filtered.T):            # calculate amino acid proportions for each column
        col_no_gaps = np.char.upper(col[col != '-'])
        aa_percs = np.array([np.sum(col_no_gaps == aa)/len(col_no_gaps) for aa in aa_order])    
        aa_prop_array[:, n_col] = aa_percs                              
    conservation_scores = np.zeros(len(columns))
    for n_col, col in enumerate(aa_prop_array.T):
        conservation_scores[n_col] = jsd(col, blosum62_background)  # compute Jensen-Shannon divergence for each column
    if include_neighbors:
        ind,dist = find_nearest(input_pdb, n_nearest)
        weights = np.array([np.exp(-decay*row) for row in dist])
        weightsums = np.sum(weights, axis = 1)
        neighbors = conservation_scores[ind]
        neighbors_weighted = np.sum(neighbors*weights, axis = 1)/weightsums
        conservation_scores_neighbors = alpha * conservation_scores + (1-alpha) * neighbors_weighted
        return conservation_scores, conservation_scores_neighbors
    else:
        return conservation_scores

In [1]:
database_fasta = './data/850db.fasta'
input_fasta = './data/AF-Q72KT1-F1.fasta'
max_seqs = 1000

In [ ]:
subprocess.run(["mmseqs","createdb",database_fasta,"./data/mmseqs_data/database_db"])                #create mmseq db file from fasta (database)
subprocess.run(["mmseqs","createdb", input_fasta, "./data/mmseqs_data/protein_db"])        #create mmseq db file from fasta (input protein)
subprocess.run(["mmseqs","search","./data/mmseqs_data/protein_db","./data/mmseqs_data/database_db","./data/mmseqs_data/result","./data/mmseqs_data/tmp","--max-seqs",str(max_seqs)])     #search homolog proteins in database
subprocess.run(["mmseqs","result2msa","./data/mmseqs_data/protein_db","./data/mmseqs_data/database_db","./data/mmseqs_data/result","./data/mmseqs_data/alignment_msa.fasta"])     # generate multiple sequence alignment from search result

In [11]:
database_fasta = './data/850db.fasta'
input_fasta = './data/AF-Q72KT1-F1.fasta'

In [ ]:
subprocess.run(["wsl", "mmseqs","createdb",database_fasta,"mmseqs/database_db"],stdout=subprocess.PIPE, text=True)

In [14]:
command = [
    "wsl", 
    "bash", "-c", 
    f"source /home/tobi/miniconda3/etc/profile.d/conda.sh && conda activate testwsl && mmseqs createdb {'/mnt/c/Users/tobia/OneDrive/Documents/Uni/splitproteins/switchable-split-proteins/data/850db.fasta'} {'mmseqs/database_db'}"
]
subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

CompletedProcess(args=['wsl', 'bash', '-c', 'source /home/tobi/miniconda3/etc/profile.d/conda.sh && conda activate testwsl && mmseqs createdb /mnt/c/Users/tobia/OneDrive/Documents/Uni/splitproteins/switchable-split-proteins/data/850db.fasta mmseqs/database_db'], returncode=1, stdout='createdb /mnt/c/Users/tobia/OneDrive/Documents/Uni/splitproteins/switchable-split-proteins/data/850db.fasta mmseqs/database_db \n\nMMseqs Version:       \t15.6f452\nDatabase type         \t0\nShuffle input database\ttrue\nCreatedb mode         \t0\nWrite lookup file     \t1\nOffset of numeric ids \t0\nCompressed            \t0\nVerbosity             \t3\n\nConverting sequences\n', stderr='Cannot open mmseqs/database_db.source for writing\n')